In [1]:
# @title Import packages and libraries

import GrfSim   # import the simulation class
import GrfStats # import the statisics analysis functions
import GrfUtils # import the utility functions

from scipy.special import hankel2
from scipy.stats import skew, kurtosis

import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
import os
import gc
import time as ti

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [2]:
# @title 1 Parameters


#RDU

# spacetimes = 'rdu'
# pixel = 2**6
# z_pixel = pixel
# z_pos = int(z_pixel//2)

# num_runs = 50

# start_time = 1
# stop_time = 40
# time_step = 1
# total_steps =  int( (stop_time - start_time) / time_step) + 1

# extra_time_range = []
# save_grf_time = [  1, 10, 20 , 40 ]
# Gaussian_time_range = [1,10]

#Desitter

"""
Spacetimes geoemtry
    'desitter' for De Sitter spacetimes
    'rdu'for radiation dominated univeres
"""
spacetimes = 'desitter'

#  Size of the simulation box (number of pixels in each dimension)
pixel = 2**6

# Set the z-dimension to be equal to x-y dimension
z_pixel = pixel

# Select the middle of the z-axis for image generation.
z_pos = int(z_pixel//2)

# Number of simulation run
num_runs = 50

# Start time for the simulation
start_time = 2e-1

# End time for the simulation
stop_time = 8

# Time step increment for the simulation
time_step = 2e-1

 # Calculate the total number of time steps
total_steps =  int( (stop_time - start_time) / time_step) + 1

 # Include additional times outside the simulation time interva
extra_time_range = [10,20]

 # Specific times to save the GRF data
save_grf_time = [ 2e-1, 4e-1, 6e-1, 8e-1, 1, 2, 4, 8, 10]

 # Specific times to plot the Gaussian probability distribution
Gaussian_time_range = [0.2, 2,10]


In [3]:
# @title 2 Setup main directory

 # Get the current working directory as main directory
main_directory = os.getcwd()

# Main path for saving simulation results based on the chosen spacetime
main_path = main_directory + f'/{spacetimes}/{pixel}'

# Create the directory if it does not exists
if not os.path.exists(main_path):
    os.makedirs(main_path)

# Change to the main directory
os.chdir(main_path)


print("Current Working Directory:", os.getcwd())

Current Working Directory: /home/k/K.Choi/RF Simulations/grf_qftcs_cosmo/desitter/64


In [ ]:
# @title 3  Run multiple simulations

"""
    Run the simulations based on the chosen spacetime.
    run_simulation_rdu for Radiation-Dominated Universe
    run_simulation_desitter for de Sitter Universe
"""

if spacetimes == 'desitter':
    print(f'Run simulations for {spacetimes} spacetimes, \nTotal run is {num_runs}, \nSimulation box size is {pixel}')
    for run_id in range(0,num_runs):
        # Initialize simulator for each run
        simulator = GrfSim.grf_sim(pixel = pixel)
        # Run simulation for de Sitter spacetime
        simulator.run_simulation_desitter(start_time = start_time,
                                 stop_time = stop_time,
                                 time_step = time_step,
                                 run_id = run_id,
                                 extra_time_range = extra_time_range,
                                 save_grf_time = save_grf_time,
                                 plot_bool = True)


if spacetimes == 'rdu':
    print(f'Run simulations for {spacetimes} spacetimes, \nTotal run is {num_runs}, \nSimulation box size is {pixel}')
    for run_id in range(0,num_runs):
        # Initialize simulator for each run
        simulator = GrfSim.grf_sim(pixel = pixel)
        # Run simulation for de Sitter spacetime
        simulator.run_simulation_rdu(start_time = start_time,
                                 stop_time = stop_time,
                                 time_step = time_step,
                                 run_id = run_id,
                                 extra_time_range = extra_time_range,
                                 save_grf_time = save_grf_time,
                                 plot_bool = True)



Run simulations for desitter spacetimes, 
Total run is 50, 
Simulation box size is 64
t= 0.2000: Computational time is: 2.04 seconds
t= 0.4000: Computational time is: 0.20 seconds
t= 0.6000: Computational time is: 0.20 seconds
t= 0.8000: Computational time is: 0.27 seconds
t= 1.0000: Computational time is: 0.20 seconds
t= 2.0000: Computational time is: 0.27 seconds
t= 4.0000: Computational time is: 0.19 seconds
t= 8.0000: Computational time is: 0.21 seconds
Run: 1: Computational time for this runs is: 10.50 seconds
Run: 2: Computational time for this runs is: 8.84 seconds


In [ ]:
# @title 4 Plot simulation statistics

# Define data prefixes and corresponding y-axis labels for plotting statistics
data_prefix_list = [ 'percent_over_2std_array', 'max_val_array', 'std_array', 'mean_array', 'skew_array', 'kurt_array']
ylabel_list = ['Percent  of samples over  Std.', 'Maximum magnitude', ' Standard deviation', 'Mean', 'Skewness', 'Kurtosis']


# Set up the path for statistics directory
folder_path = main_path + '/statistics'

# Create the directory if it does not exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Loop over each data type and plot the corresponding statistics
for data_prefix, ylabel in zip(data_prefix_list, ylabel_list):

    # Change to the main directory
    os.chdir(main_path)
    # Plot the statistics
    figures = GrfStats.plot_multiple_runs( data_prefix, ylabel, total_steps, time_step)

    # Change to the statistics directory
    os.chdir(folder_path)
    # Save the plot as a PDF file
    figures.savefig(f'{data_prefix}_pixel={pixel}_run={num_runs}.pdf')

In [ ]:
# @title 5 Plot (Gaussian) Probability density distribution

"""
    Plot the Gaussian probability distribution for the field at specified times.
"""

# Set up the path for distribution directory
folder_path = main_path + '/distributions'

# Create the directory if it does not exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

fig = plt.figure(figsize=(8, 6))


# Loop over the specified times and plot the Gaussian probability distribution
for time in Gaussian_time_range:

    # Change to the main directory
    os.chdir(main_path)

    # Load Guassian Random Field data
    grf = np.load(f'grf_t={time:.4f}_pixel={pixel}.npy')

    # Change to the distributions directory
    os.chdir(folder_path)
    # Plot the histogram
    count, bins, _ = plt.hist( grf.flatten() , bins=501, alpha=0.25, color='blue', density=True)
    bin_centers = 0.5* ( bins[1:] + bins[:-1] )
     # Overlay a line on the edge of the histogram
    plt.plot( bin_centers, count, '--', color= 'red', linewidth =1)

# Set plot limits and labels
plt.xlim(-1.1, 1.1)
plt.tick_params(axis='both', colors='black', direction='out', length=4, width=1, labelsize=18)
plt.xlabel('Field amplitude', fontsize=18, color='black')
plt.ylabel('Probability density', fontsize=18, color='black')

# Save the Gaussian probability density plot as a PDF
plt.savefig(f'grf_gaussian_g={spacetimes}.pdf', bbox_inches = 'tight',pad_inches = 0)


"""
    Plot the Half normal probability distribution for the field magitude at the specified times.
"""

fig = plt.figure(figsize=(8, 6))

# Loop over the specified times and plot the half normal probability distribution
for time in Gaussian_time_range:

    # Change to the main directory
    folder_path = main_path + '/distributions'
    os.chdir(main_path)

    # Load Guassian Random Field data
    grf = np.load(f'grf_t={time:.4f}_pixel={pixel}.npy')

    # Change to the distributions directory
    os.chdir(folder_path)

    # Plot the histogram
    count, bins, _ = plt.hist( (np.abs(grf)).flatten() , bins=501, alpha=0.25, color='blue', density=True)
    bin_centers = 0.5* ( bins[1:] + bins[:-1] )
    # Overlay a line on the edge of the histogram
    plt.plot( bin_centers, count, '--', color= 'red', linewidth = 1)

# Set plot limits and labels
plt.xlim(0, 1.1)
plt.tick_params(axis='both', colors='black', direction='out', length=6, width=2, labelsize=18)
plt.xlabel('Field magnitude', fontsize=18, color='black')
plt.ylabel('Probability density', fontsize=18, color='black')

# Save the half normal probability density plot as a PDF
plt.savefig(f'grf_half_normal_g={spacetimes}.pdf', bbox_inches = 'tight',pad_inches = 0)


In [ ]:
# @title 6 Plot 2D Random field configurations


"""
    Plotting 2D Guassian Radnom Field configurations with amplitude and magnitude.
"""


# Set up the path for configuration directory
folder_path = main_path + f'/field_configs'

# Create the directory if it does not exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Loop over the specified times and plot the 2D random field configurations
for time in save_grf_time:

    # Change to the main directory
    os.chdir(main_path)

    # Load Guassian Random Field data
    grf = np.load(f'grf_t={time:.4f}_pixel={pixel}.npy')

    # Change to the configuration directory
    os.chdir(folder_path)

    # Plot the 2D random field configurations ampltiude
    grf_plt = GrfStats.plot_grf_2d(grf)
    # Plot the 2D random field configurations magitude
    grf_plt_linear_hot = GrfStats.plot_grf_2d(grf = np.abs(grf), min_val=0, max_val = 1, cmap='inferno' , normalization=False)
    # Plot the 2D random field configurations magitude with logscale color scheme
    grf_plt_hot = GrfStats.logplot_grf_2d(np.abs(grf), normalization=False)

    # Release memories
    del grf
    gc.collect()
    plt.close('all')

    # Save the plots as PNG and PDF files
    grf_plt.savefig(f'grf_map_g={spacetimes}_t={time:.4f}.png', bbox_inches = 'tight',pad_inches = 0.05)
    grf_plt.savefig(f'grf_map_g={spacetimes}_t={time:.4f}.pdf', bbox_inches = 'tight',pad_inches = 0.05)
    grf_plt_hot.savefig(f'grf_modulus_map_g={spacetimes}_t={time:.4f}.png', bbox_inches = 'tight',pad_inches = 0.05)
    grf_plt_hot.savefig(f'grf_modulus_map_g={spacetimes}_t={time:.4f}.pdf', bbox_inches = 'tight',pad_inches = 0.05)
    grf_plt_linear_hot.savefig(f'grf_linear_modulus_map_g={spacetimes}_t={time:.4f}.png', bbox_inches = 'tight',pad_inches = 0.05)
    grf_plt_linear_hot.savefig(f'grf_linear_modulus_map_g={spacetimes}_t={time:.4f}.pdf', bbox_inches = 'tight',pad_inches = 0.05)

In [ ]:
 # @title 7 Plot statistics of the 2D Random field configurations

# Set up the path for configuration directory
folder_path = main_path + f'/field_configs'

# Create the directory if it does not exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Loop over the specified times and plot the statistics of the 2D random field configurations
for time in save_grf_time:

    # Change to the main directory
    os.chdir(main_path)

    # Load Guassian Random Field data
    grf = np.load(f'grf_t={time:.4f}_pixel={pixel}.npy')

    # Change to the configuration directory
    os.chdir(folder_path)

    # Calculate statistics of the 2D random field configurations
    stat_fig, _, _, _, _ = GrfStats.stats_overview(grf)

    # Release memories
    del grf
    gc.collect()
    plt.close('all')

    # Save the statistics plot as PNG and PDF files
    stat_fig.savefig(f'grf_statistics_g={spacetimes}_t={time:.4f}.pdf', bbox_inches = 'tight',pad_inches = 0.05)
    stat_fig.savefig(f'grf_statistics_g={spacetimes}_t={time:.4f}.pdf', bbox_inches = 'tight',pad_inches = 0.05)

In [ ]:
# @title Print legends

"""
    Printing colorbor only.
"""

os.chdir(main_path)
GrfUtils.create_colorbar(0.01, 1.0, 'inferno', 'inferno_legend.png')
GrfUtils.create_colorbar(-1.0, 1.0, 'viridis', 'viridis_legend.png')
GrfUtils.create_log_colorbar(0.01, 1.0, 'inferno', 'log_inferno_legend.png')